# Collect Data for Test 2

In [ ]:
import pandas as pd
from datetime import datetime
from data_collector import collect_data_inkl_news, determine_trend, get_news, sentiment_int
import os
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands


In [ ]:
symbol = "SPY"
start_year = 2015
end_year = 2023

## Training Data

In [ ]:
periods = [(datetime(year, 1, 1), datetime(year, 12, 31)) for year in range(start_year, end_year)]

folder_path = "../../Data/spy_data"
os.makedirs(folder_path, exist_ok=True)

for start, end in periods:
    df = collect_data_inkl_news(
    symbol=symbol,
    start=start,
    end=end,
    alpaca_symbol="SPY" 
    )
    df.to_csv(f"{folder_path}/spy_{start}-{end}.csv")
    
    


In [ ]:
folder_path = "../../Data/"
spy_files = os.listdir(f"{folder_path}spy_data")

df_list = []
for file in spy_files:
    if file == "spy_concat.csv":
        continue
    df = pd.read_csv(f"{folder_path}spy_data/{file}")
    df.drop(columns=["Unnamed: 0"], inplace=True)
    df_list.append(df)

df = pd.concat(df_list)

df.reset_index(inplace=True)
df = df[["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume", "month", "weekday", "news_probability", "news_sentiment"]]
df["news_probability"] = df["news_probability"].apply(lambda x: float(x.removeprefix("tensor(").split(",")[0]))

df["trend"] = df.apply(determine_trend, axis=1)
df["sentiment_int"] = df.apply(sentiment_int, axis=1)
df = df.drop(columns=["news_sentiment"])

df['rsi'] = RSIIndicator(df['Close'], window=14).rsi()
macd = MACD(close=df['Close'])
df['macd'] = macd.macd()
df['macd_signal'] = macd.macd_signal()
bollinger = BollingerBands(close=df['Close'], window=20)
df['bollinger_hband'] = bollinger.bollinger_hband()
df['bollinger_lband'] = bollinger.bollinger_lband()

df["moving_avg"] = None
df["Y"] = None

df["Y"] = df["Close"].shift(-1)
df["moving_avg"] = df["Close"].shift(1).rolling(window=3).mean()
df.dropna(inplace=True)

df["month"] = df["month"].astype(int)
df["weekday"] = df["weekday"].astype(int)

df.to_pickle(f"{folder_path}train_spy_data.pkl")
display(df)



## Test Data

In [ ]:
folder_path = "../../Data/"
os.makedirs(f"{folder_path}test_data", exist_ok=True)

df = collect_data_inkl_news(
    symbol=symbol,
    start=datetime(2022, 8, 1),
    end=datetime(2024, 1, 31),
    alpaca_symbol="SPY" 
)
df.to_csv(f"{folder_path}/spy.csv")

In [ ]:
df = pd.read_csv(f"{folder_path}/spy.csv")

df.reset_index(inplace=True)
df = df[["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume", "month", "weekday", "news_probability", "news_sentiment"]]
df["news_probability"] = df["news_probability"].apply(lambda x: float(x.removeprefix("tensor(").split(",")[0]))

df["trend"] = df.apply(determine_trend, axis=1)
df["sentiment_int"] = df.apply(sentiment_int, axis=1)
df = df.drop(columns=["news_sentiment"])

df['rsi'] = RSIIndicator(df['Close'], window=14).rsi()
macd = MACD(close=df['Close'])
df['macd'] = macd.macd()
df['macd_signal'] = macd.macd_signal()
bollinger = BollingerBands(close=df['Close'], window=20)
df['bollinger_hband'] = bollinger.bollinger_hband()
df['bollinger_lband'] = bollinger.bollinger_lband()

df["moving_avg"] = None
df["Y"] = None

df["Y"] = df["Close"].shift(-1)
df["moving_avg"] = df["Close"].shift(1).rolling(window=3).mean()
df.dropna(inplace=True)

df["month"] = df["month"].astype(int)
df["weekday"] = df["weekday"].astype(int)

df.to_pickle(f"{folder_path}test_spy_data.pkl")
display(df)